# ДЗ по "Сбор и разметка данных (семинары)"
## Семинар 2. Парсинг HTML. BeautifulSoup
Домашнее задание:
Выполнить скрейпинг данных в веб-сайта http://books.toscrape.com/ и извлечь информацию о всех книгах на сайте во всех категориях: название, цену, количество товара в наличии (In stock (19 available)) в формате integer, описание. Затем сохранить эту информацию в JSON-файле

In [1]:
# Устанавливаем библиотеки, если они не установлены
# !pip install pandas
# !pip install regex

In [2]:
# Импорт необходимых библиотек
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.parse
import re
import time
import json

In [3]:
# Пауза при отправке запроса на сервер
TIME_SLEEP = 0.025

# Путь к странице с данным
website = "https://books.toscrape.com/"
# Стартовая страница сайта
url_page = 'index.html'

In [4]:
# Создаем сессию, чтобы делать запрос из одной сессии
session = requests.session()
# Заголовок
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

## Проведем скрейпинг ссылок на категории книг

In [5]:
# GET-запрос к серверу
page = session.get(urllib.parse.urljoin(website, url_page), headers=headers)
# Парсинг данных
soup = BeautifulSoup(page.content, 'html.parser')
time.sleep(TIME_SLEEP)

In [6]:
# Парсинг категории книг
book_category_list = soup.find('ul', ('class', 'nav nav-list')).find_all('a')
# Создадим словарь для категорий книг
book_category_dict = {}
for item in book_category_list[1:]:
    # Записываем название категории и ссылку на категорию
    book_category_dict[item.get_text().strip()] = urllib.parse.urljoin(website, item.get('href'))
    


In [7]:
book_category_dict


{'Travel': 'https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'Mystery': 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'Historical Fiction': 'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'Sequential Art': 'https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'Classics': 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'Philosophy': 'https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'Romance': 'https://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'Womens Fiction': 'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'Fiction': 'https://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'Childrens': 'https://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'Religion': 'https://books.toscrape.com/catalogue/categor

## Проведем скрейпинг ссылок на все книги

In [8]:
#Словарь всех книг во всех категориях
all_book = {}

for category, page_category in book_category_dict.items():
    book_in_category = []
    # Число книг в категории
    count_book_in_category = 0
    while True:       
        # GET-запрос к серверу
        page = session.get(page_category, headers=headers)
        # Парсинг данных
        soup = BeautifulSoup(page.content, 'html.parser')
        time.sleep(TIME_SLEEP)
         
        # Парсим книги на странице
        category_list = soup.find_all('article', ('class', 'product_pod'))       
        for item in category_list:
            # Парсим ссылку на книгу и добавляем полную версию ссылки в список
            *_, folder, file = item.find('h3').find('a').get('href').split('/') 
            full_url = urllib.parse.urljoin(website, f'/catalogue/{folder}/{file}')
            book_in_category.append(full_url)
            count_book_in_category += 1
          
        # Проверяем есть ли кнопка 'Next' на странице
        next_page_link = soup.find('li', ('class', 'next'))
        if next_page_link:
            # Записываем текущий каталог
            *_, dir_name = page_category.split('/')
            len_file_name = len(dir_name)
            p_folder = str(page_category)[:-len_file_name]
            # Записываем полный адрес
            page_category = urllib.parse.urljoin(p_folder, next_page_link.find('a').get('href'))
        else:               
            # Добавляем ссылки на книги в текущую категорию    
            all_book[category] = book_in_category
            print(f'В категории "{category}" = {count_book_in_category} книг.')
            break

В категории "Travel" = 11 книг.
В категории "Mystery" = 32 книг.
В категории "Historical Fiction" = 26 книг.
В категории "Sequential Art" = 75 книг.
В категории "Classics" = 19 книг.
В категории "Philosophy" = 11 книг.
В категории "Romance" = 35 книг.
В категории "Womens Fiction" = 17 книг.
В категории "Fiction" = 65 книг.
В категории "Childrens" = 29 книг.
В категории "Religion" = 7 книг.
В категории "Nonfiction" = 110 книг.
В категории "Music" = 13 книг.
В категории "Default" = 152 книг.
В категории "Science Fiction" = 16 книг.
В категории "Sports and Games" = 5 книг.
В категории "Add a comment" = 67 книг.
В категории "Fantasy" = 48 книг.
В категории "New Adult" = 6 книг.
В категории "Young Adult" = 54 книг.
В категории "Science" = 14 книг.
В категории "Poetry" = 19 книг.
В категории "Paranormal" = 1 книг.
В категории "Art" = 8 книг.
В категории "Psychology" = 7 книг.
В категории "Autobiography" = 9 книг.
В категории "Parenting" = 1 книг.
В категории "Adult Fiction" = 1 книг.
В катег

## Проведем скрейпинг информации о каждой книги (категория, название, цена и т.д.) 

In [9]:
book_count = 0
result_list = []
for category, link_book_list in all_book.items():
    for link in link_book_list:
        temp_dic = {}
        # GET-запрос к серверу
        page = session.get(link, headers=headers)
        # Парсинг данных
        soup = BeautifulSoup(page.content, 'html.parser')
        time.sleep(TIME_SLEEP)
        # Категория книги
        temp_dic['category'] = category
        product_main = soup.find('div', ('class', 'col-sm-6 product_main'))
        # Название книги
        name = product_main.find('h1').getText(strip=True)
        temp_dic['name'] = name
        # Цена
        price = product_main.find('p', ('class', 'price_color')).getText(strip=True)
        price = price.replace(',', '.')
        try:
            temp_dic['price'] = float(re.sub(r'[^\d.]+', '', price))
        except ValueError:
            temp_dic['price'] = None
        # Наличие
        available = product_main.find('p', ('class', 'instock availability')).getText(strip=True)
        try:
            temp_dic['available'] = int(re.sub(r'[^\d.]+', '', available))
        except ValueError:
            temp_dic['available'] = None
        # Описание книги
        description = soup.find_all('p')
        temp_dic['description'] = description[3].getText()
        result_list.append(temp_dic)
        book_count += 1
        # Выведем на печать, чтобы было понятно, как процесс движется.
        print(f'{book_count:<5}  {name} {link}')
        time.sleep(TIME_SLEEP)

1      It's Only the Himalayas https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html
2      Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html
3      See America: A Celebration of Our National Parks & Treasured Sites https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html
4      Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel https://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html
5      Under the Tuscan Sun https://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html
6      A Summer In Europe https://books.toscrape.com/catalogue/a-summer-in-europe_458/index.html
7      The Great Railway Bazaar https://books.toscrape.com/catalogue/the-great-railway-bazaar_446/index.html
8      A Year in

In [10]:
df = pd.DataFrame(result_list)

In [11]:
df.head()

,category,name,price,available,description
0,Travel,It's Only the Himalayas,45.17,19,"“Wherever you go, whatever you do, just . . . ..."
1,Travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,15,Acclaimed travel writer Rick Antonson sets his...
2,Travel,See America: A Celebration of Our National Par...,48.87,14,To coincide with the 2016 centennial anniversa...
3,Travel,Vagabonding: An Uncommon Guide to the Art of L...,36.94,8,With a new foreword by Tim Ferriss •There’s no...
4,Travel,Under the Tuscan Sun,37.33,7,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...


In [12]:
df.shape

(1000, 5)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   category     1000 non-null   object 
 1   name         1000 non-null   object 
 2   price        1000 non-null   float64
 3   available    1000 non-null   int64  
 4   description  1000 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 39.2+ KB


## Сохраняем полученную информацию в JSON файл

In [14]:
with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result_list, f, ensure_ascii=False)
    print('Запись выполнена')

Запись выполнена


# Вывод:
## Выполнил скрейпинг веб-сайта [http://books.toscrape.com](http://books.toscrape.com), сохранил информацию о всех книгах (1_000 шт.) в JSON файл. Использовал в разработке Jupiter notebook в IDE pycharm professional (поставил ознакомительную версию на 30 дней). В самом Jupiter notebook нет среды отладки – это я считаю большим минусом, хотя IDE очень классная.
## Еще один момент заметил, если залить файлы, созданные в IDE pycharm professional на GitHub, файлы не просматриваются, написано, что старая версия. Надо пересохранять в обычном  Jupiter notebook.